In [1]:
import tkinter
from tkinter import ttk
import psycopg2


def connect_to_postgres():
    try:
        conn = psycopg2.connect(
            database="student_data",  # Replace with your database name
            user="postgres",      # Replace with your PostgreSQL username
            password="123456",  # Replace with your PostgreSQL password
            host="localhost",      # Replace with your PostgreSQL host (e.g., "localhost" or an IP address)
            port="5432"       # Replace with your PostgreSQL port (default is 5432)
        )
        return conn
    except psycopg2.Error as e:
        print("Error connecting to PostgreSQL:", e)
        return None
    
def create_student_data_database():
    try:
        # Connect to PostgreSQL as a superuser or a user with database creation privileges
        conn = connect_to_postgres()
        if conn is None:
            return

        # Create a cursor
        cur = conn.cursor()

        # Create the "student_data" database if it doesn't exist
        cur.execute("CREATE DATABASE IF NOT EXISTS student_data")
        print("Database 'student_data' created successfully")

    except psycopg2.Error as e:
        print("Error creating 'student_data' database:", e)
        conn.rollback()
    finally:
        if conn is not None:
            conn.close()

# Call the function to create the "student_data" database
create_student_data_database()


def enter_data():
    # User info
    firstname = first_name_entry.get()
    lastname = last_name_entry.get()
    title = title_combobox.get()
    age = age_spinbox.get()
    nationality = nationality_combobox.get()

    # Display a success message
    success_label.config(text="Your data is recorded")

    # Courses info
    registration_status = reg_status_var.get()
    numcourses = numcourses_spinbox.get()
    numsemesters = numsemesters_spinbox.get()

    try:
        # Connect to the "student_data" database
        conn = psycopg2.connect(
            database="student_data",
            user="your_database_user",  # Replace with the appropriate PostgreSQL user
            password="your_database_password",  # Replace with the appropriate password
            host="your_database_host",  # Replace with the appropriate host
            port="your_database_port"  # Replace with the appropriate port
        )

        # Create a cursor
        cur = conn.cursor()

        # Create a table if it doesn't exist
        cur.execute('''
            CREATE TABLE IF NOT EXISTS user_data (
                id SERIAL PRIMARY KEY,
                first_name TEXT,
                last_name TEXT,
                title TEXT,
                age INTEGER,
                nationality TEXT,
                registration_status TEXT,
                num_courses INTEGER,
                num_semesters INTEGER
            )
        ''')

    
        # Insert the data into the table
        cur.execute('''
            INSERT INTO user_data (
                first_name, last_name, title, age, nationality,
                registration_status, num_courses, num_semesters
            )
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        ''', (firstname, lastname, title, age, nationality, registration_status, numcourses, numsemesters))

        # Commit the changes
        conn.commit()

        # Display a success message
        success_label.config(text="Your data is recorded")

        print("First name:", firstname, "Last Name:", lastname)
        print("Title:", title, "Age:", age, "Nationality:", nationality)
        print("# Courses:", numcourses, "# Semesters:", numsemesters)
        print("Registration status", registration_status)
        print("----------------------------------")

    except psycopg2.Error as e:
        # Handle any exceptions, e.g., database errors
        print("Error:", str(e))
    finally:
        if conn is not None:
            conn.close()

window = tkinter.Tk()
window.title("Data Entry Form")

frame = tkinter.Frame(window)
frame.pack()

#saving User information
user_info_frame = tkinter.LabelFrame(frame, text= "User Information")
user_info_frame.grid(row=0,column=0, padx=20, pady=20)

first_name_label = tkinter.Label(user_info_frame, text= "FIrst Name")
first_name_label.grid(row=0, column=0)
last_name_label=tkinter.Label(user_info_frame, text="Last Name")
last_name_label.grid(row=0, column=1)

first_name_entry = tkinter.Entry(user_info_frame)
last_name_entry = tkinter.Entry(user_info_frame)
first_name_entry.grid(row=1,column=0)
last_name_entry.grid(row=1, column=1)

title_label = tkinter.Label(user_info_frame,text="Title")
title_combobox = ttk.Combobox(user_info_frame, values = ["Mr.","Ms.","Mrs.","Dr."])
title_label.grid(row=0, column=2)
title_combobox.grid(row=1, column=2)

age_label = tkinter.Label(user_info_frame, text = "Age")
age_spinbox = tkinter.Spinbox(user_info_frame)
age_label.grid(row=2, column=0)
age_spinbox.grid(row=3, column=0)

nationality_label =tkinter.Label(user_info_frame, text = "Nationality")
nationality_combobox= ttk.Combobox(user_info_frame, values=["Africa", "Antarctica", "Asia", "Europe", "America"])
nationality_label.grid(row=2, column=1)
nationality_combobox.grid(row=3, column=1)

for widget in user_info_frame.winfo_children():
    widget.grid_configure(padx=10, pady=5)

#Saving COurse Info
courses_frame = tkinter.LabelFrame(frame)
courses_frame.grid(row=1, column=0, sticky= "news", padx=20, pady=10)

registered_label =tkinter.Label(courses_frame, text = "Registration Status")

reg_status_var = tkinter.StringVar()
registered_check = tkinter.Checkbutton(courses_frame, text = "Currently Registered", 
                                       variable=reg_status_var)

registered_label.grid(row=0, column=0)
registered_check.grid(row=1, column=0)

numcourses_label = tkinter.Label(courses_frame, text = "# Completed Courses")
numcourses_spinbox = tkinter.Spinbox(courses_frame, from_=0, to ="infinity")
numcourses_label.grid(row=0, column= 1)
numcourses_spinbox.grid(row=1, column=1)

numsemesters_label = tkinter.Label(courses_frame, text = "# Semesters")
numsemesters_spinbox = tkinter.Spinbox(courses_frame, from_=0, to ="infinity")
numsemesters_label.grid(row=0, column= 2)
numsemesters_spinbox.grid(row=1, column=2)

for widget in courses_frame.winfo_children():
    widget.grid_configure(padx=10, pady=5)

#Accept terms and conditions
terms_frame = tkinter.LabelFrame(frame, text= "Terms & Conditions")
terms_frame.grid(row=2, column=0, sticky= "news", padx=20, pady= 10)

terms_check = tkinter.Checkbutton(terms_frame, text= "I accept the terms and conditions")
terms_check.grid(row=0, column=0)

# Add a label for displaying the success message
success_label = tkinter.Label(frame, text="", fg="green")
success_label.grid(row=4, column=0, padx=20, pady=10)

# Button
button = tkinter.Button(frame, text="Enter data", command=enter_data)
button.grid(row=3, column=0, sticky="news", padx=20, pady=10)

#Add a close button
close_button = tkinter.Button(frame, text = "Close", command=window.destroy)
close_button.grid(row=4, column=0, sticky="news",padx=20, pady=10)

window.mainloop()

Error connecting to PostgreSQL: connection to server at "localhost" (::1), port 5432 failed: FATAL:  database "student_data" does not exist

Error: invalid integer value "your_database_port" for connection option "port"



Exception in Tkinter callback
Traceback (most recent call last):
  File "F:\Users\cvpri\anaconda3\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\cvpri\AppData\Local\Temp\ipykernel_12520\367070207.py", line 115, in enter_data
    if conn is not None:
       ^^^^
UnboundLocalError: cannot access local variable 'conn' where it is not associated with a value
